A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class Net(nn.Module):
    def __init__(self):
        # inherit from parent class
        super(Net, self).__init__()
        # define self
        # kernel: input, output, kernel size (5x5 square convolution)
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # flatten all dimensions except the batch dimension
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    # backward function is automatically defined using autograd

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
# The learnable parameters of a model are returned by net.parameters()
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params[1].size())  # conv1's .bias

10
torch.Size([6, 1, 5, 5])
torch.Size([6])


In [5]:
# try a random 32x32 input.
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0290,  0.1507, -0.0111, -0.0270, -0.1634, -0.0729, -0.0216,  0.1128,
          0.0079,  0.0091]], grad_fn=<AddmmBackward0>)


In [6]:
# call backward
net.zero_grad()
out.backward(torch.randn(1, 10))

In [7]:
# Loss Function
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.5527, grad_fn=<MseLossBackward0>)


In [8]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# Print Structure
def print_graph(g, level=0):
    if g == None: return
    print('*'*level*4, g)
    for subg in g.next_functions:
        print_graph(subg[0], level+1)

print_graph(loss.grad_fn, 0)

**** <AddmmBackward0 object at 0x7f099cee8f70>
******** <AccumulateGrad object at 0x7f099ce84070>
******** <ReluBackward0 object at 0x7f099ce840a0>
************ <AddmmBackward0 object at 0x7f099ce844f0>
**************** <AccumulateGrad object at 0x7f099ce846d0>
**************** <ReluBackward0 object at 0x7f099ce84700>
******************** <AddmmBackward0 object at 0x7f099ce84a90>
************************ <AccumulateGrad object at 0x7f099ce84c70>
************************ <ReshapeAliasBackward0 object at 0x7f099ce84ca0>
**************************** <MaxPool2DWithIndicesBackward0 object at 0x7f099ce85070>
******************************** <ReluBackward0 object at 0x7f099ce85250>
************************************ <ConvolutionBackward0 object at 0x7f099ce85430>
**************************************** <MaxPool2DWithIndicesBackward0 object at 0x7f099ce85610>
******************************************** <ReluBackward0 object at 0x7f099ce85850>
***********************************************

In [9]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0066,  0.0110,  0.0219, -0.0128, -0.0139, -0.0064])


In [10]:
# Update Learning Rate
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [11]:
# Enable different update rules by torch.optim
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update